In [1]:
#pip install "altair[all]"

In [2]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

In [3]:
crime2022_df = pd.read_csv('ProjectData/crime_reports_2022.csv', low_memory=False)
crime2021_df = pd.read_csv('ProjectData/2021.csv', low_memory=False)
crime2020_df = pd.read_csv('ProjectData/2020.csv', low_memory=False)
crime2019_df = pd.read_csv('ProjectData/2019.csv', low_memory=False)
crime2022_df.head(100)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,222076257,619,NaN,LARCENY ALL OTHERS,D4,167,0,2022-01-01 00:00:00,2022,1,Saturday,0,NaN,HARRISON AVE,42.339542,-71.069409,"(42.33954198983014, -71.06940876967543)"
1,222053099,2670,NaN,HARASSMENT/ CRIMINAL HARASSMENT,A7,,0,2022-01-01 00:00:00,2022,1,Saturday,0,NaN,BENNINGTON ST,42.377246,-71.032597,"(42.37724638479816, -71.0325970804128)"
2,222039411,3201,NaN,PROPERTY - LOST/ MISSING,D14,778,0,2022-01-01 00:00:00,2022,1,Saturday,0,NaN,WASHINGTON ST,42.349056,-71.150498,"(42.34905600030506, -71.15049849975023)"
3,222011090,3201,NaN,PROPERTY - LOST/ MISSING,B3,465,0,2022-01-01 00:00:00,2022,1,Saturday,0,NaN,BLUE HILL AVE,42.284826,-71.091374,"(42.28482576580488, -71.09137368938802)"
4,222062685,3201,NaN,PROPERTY - LOST/ MISSING,B3,465,0,2022-01-01 00:00:00,2022,1,Saturday,0,NaN,BLUE HILL AVE,42.284826,-71.091374,"(42.28482576580488, -71.09137368938802)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,222068405,1102,NaN,FRAUD - FALSE PRETENSE / SCHEME,B2,282,0,2022-01-01 10:00:00,2022,1,Saturday,10,NaN,WASHINGTON ST,42.328663,-71.085634,"(42.32866283555608, -71.08563401183545)"
96,222000102,724,NaN,AUTO THEFT,A7,,0,2022-01-01 10:08:00,2022,1,Saturday,10,NaN,PRESCOTT ST,42.378741,-71.029611,"(42.37874084446998, -71.02961119358292)"
97,222000103,3802,NaN,M/V ACCIDENT - PROPERTY DAMAGE,D14,,0,2022-01-01 10:08:00,2022,1,Saturday,10,NaN,COLBORNE RD & RANSOM RD\nBRIGHTON MA 02135\nU...,42.342515,-71.146169,"(42.342514954114826, -71.14616895804701)"
98,222000860,2647,NaN,THREATS TO DO BODILY HARM,B2,320,0,2022-01-01 10:15:00,2022,1,Saturday,10,NaN,HARTWELL ST,42.309364,-71.084168,"(42.30936380087264, -71.08416777254996)"


In [4]:
crime2023_present = pd.read_csv('ProjectData/2023_to_present.csv', low_memory=False)
crime2023_df = crime2023_present.loc[crime2023_present["OCCURRED_ON_DATE"].between("2023-01-01", "2023-12-31")]
crime2024_df = crime2023_present.loc[crime2023_present["OCCURRED_ON_DATE"].between("2024-01-01", "2024-12-31")]

In [5]:
# Dataframe containing the key for offense codes marked in the data
offense_codes = pd.read_excel('ProjectData/rmsoffensecodes.xlsx')
result = offense_codes[offense_codes['CODE'] == 3115]
#offense_codes.head(100) 
result

,CODE,NAME
151,3115,INVESTIGATE PERSON
427,3115,INVESTIGATE PERSON


In [19]:
def get_monthly_groupings_counts(years_df, columns_to_group_on):
    df = years_df.copy()
    df['OCCURRED_ON_DATE'] = pd.to_datetime(df['OCCURRED_ON_DATE'], format='ISO8601')
    df['month_year'] = df['OCCURRED_ON_DATE'].dt.strftime('%Y-%m')

    df['year'] = df["month_year"].str[:4]
    df['month'] = df["month_year"].str[5:].astype(int)

    # Reformat so the month number is actually the written month
    df['month'] = pd.to_datetime(df['month'], format='%m').dt.strftime('%b')
    # Define the correct month order I'm using
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    # Convert 'Month' column to categorical type with correct order
    df['month'] = pd.Categorical(df['month'], categories=month_order, ordered=True)
    # Re-sort values to be the correct month order
    df = df.sort_values('month')

    counts_df = df.groupby(columns_to_group_on).size().reset_index(name='count')

    return counts_df

In [20]:
# Make df with all years
all_years = pd.concat([crime2024_df, crime2023_df, crime2022_df, crime2021_df, crime2020_df, crime2019_df])

In [21]:
# Run groupby function for month/year
six_year_totals = get_monthly_groupings_counts(all_years, ['year', 'month'])
six_year_totals

/var/folders/g2/pgy578q902gdqfb874mvp60h0000gn/T/ipykernel_48576/1606314915.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  counts_df = df.groupby(columns_to_group_on).size().reset_index(name='count')


,year,month,count
0,2019,Jan,6958
1,2019,Feb,6173
2,2019,Mar,7093
3,2019,Apr,6963
4,2019,May,7783
...,...,...,...
67,2024,Aug,7087
68,2024,Sep,7019
69,2024,Oct,6792
70,2024,Nov,6473


In [ ]:
# Chart showing the total count of offenses recorded in Boston for each month by year
# Month order to make sure the data is in the proper order
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
total_offense_line = alt.Chart(four_year_totals).mark_line().encode(
    x = alt.X('month:N', sort = month_order).title('Month'),
    y = alt.Y('count:Q').title('Total Offense Counts'),
    color = alt.Color('year:N', title = 'Year'),
    tooltip = ['year', 'month', 'count']
).properties(
    title = 'Total Offense Counts in Boston by Year',
    width = 600,
    height = 400
)
total_offense_line.save('yearly_offenses.html')
total_offense_line

In [ ]:
# Goal: Showing the different offense totals that occur between two years
two_years = pd.concat([crime2022_df, crime2021_df])
two_year_offenses = get_monthly_groupings_counts(two_years, ['year', 'month', 'OFFENSE_CODE'])
# Set offense codes to be strings to be used in altair modeling
two_year_offenses["OFFENSE_CODE"] = two_year_offenses["OFFENSE_CODE"].astype(str)
two_year_offenses = two_year_offenses.groupby(['year', 'month', 'OFFENSE_CODE'], as_index=False)['count'].sum()
two_year_offenses

In [ ]:
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

stacked_bar = alt.Chart(two_year_offenses).mark_bar().encode(
    x = alt.X('year:N').title('Year'),
    y = alt.Y('count:Q').title('Total Monthly Offense Count'),
    color = alt.Color('OFFENSE_CODE:N', title='Offense Code', legend=alt.Legend(title='Offense Code', symbolLimit=10)),
    facet=alt.Facet('month:N', title='month', sort=month_order, columns=6),
    tooltip=['year:N', 'month:N', 'OFFENSE_CODE:N', 'count:Q']
).properties(
    width=100,
    height=300,
    title="Monthly Stacked Bar Chart of Offense Counts by Year"
)
stacked_bar.save('Offense_stacked_bar.html')

stacked_bar